# Match BGBM Collectors to Wikidata Items Using *Nearest Neighbour*, Just Name Comparison

See also [match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_k-nearest.ipynb](./match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_k-nearest.ipynb) that includes the `eventDate` of sampling to get a life time reference for deciding name matches to WikiData.

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

For the output of DarwinCore agent attribution, reconsidere `displayOrder` that it represents rather the data quality first and foremost, *not* the very name match.

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv(
    "data/wikidata_persons_botanists_20230703_1352.csv", 
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
pprint.pprint(wikidata.columns)
display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)

# colls = list(wikidata.columns)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


### Load Collectors Data Set

Data sources:

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

In [3]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.
collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample
3,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano,E.",parsed:E. A. Cano,cleaned:E. A. Cano,1,https://herbarium.bgbm.org/object/B100699397
39913,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murashige & Aaiki,parsed:Murashige<SEP>Aaiki,cleaned:Murashige<SEP>Aaiki,1,https://herbarium.bgbm.org/object/B101149305
7,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn,A.",parsed:A. Aaronsohn,cleaned:A. Aaronsohn,3,https://je.jacq.org/JE00010154
27133,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M.Ait Lafki...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,3,https://herbarium.bgbm.org/object/B100217620
27137,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A., Ait Lafkih,M. & Griffith...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Lafk...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,2,https://herbarium.bgbm.org/object/B100326682
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66765,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova,O.",parsed:O. Ždanova,cleaned:O. Ždanova,5,https://herbarium.bgbm.org/object/B100263330
32999,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Lomonosova,M., Ždanova,O. & Šaulo,D.",parsed:M. Lomonosova<SEP>O. Ždanova<SEP>D. Šaulo,cleaned:M. Lomonosova<SEP>O. Ždanova<SEP>D. Šaulo,1,https://herbarium.bgbm.org/object/B100263331
66766,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla,V.",parsed:V. Žíla,cleaned:V. Žíla,3,https://herbarium.bgbm.org/object/B100009590
66767,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,Волкова Е.,parsed:Е. Волкова,cleaned:Е. Волкова,1,https://herbarium.bgbm.org/object/B100530714


#### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (1364 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample
2327,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981
27224,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021
21177,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...
18599,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455
417,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C.<SEP>Al,cleaned:Schiers Ahagen<SEP>C.<SEP>Al,1,https://herbarium.bgbm.org/object/B100194646
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65971,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath,Ernst",parsed:Ernst Zickendrath,cleaned:Ernst Zickendrath,3,https://je.jacq.org/JE04006629
65972,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath,Ernst & Heyden,K.L.",parsed:Ernst Zickendrath<SEP>K.L. Heyden,cleaned:Ernst Zickendrath<SEP>K.L. Heyden,1,https://je.jacq.org/JE04007139
66058,Ziz,Johann Baptist,NaN,NaN,NaN,NaN,NaN,NaN,"Ziz,Johann Baptist",parsed:Johann Baptist Ziz,cleaned:Johann Baptist Ziz,1,https://je.jacq.org/JE00017744
66120,Zollinger,Heinrich,NaN,NaN,NaN,NaN,NaN,NaN,"Zollinger,Heinrich",parsed:Heinrich Zollinger,cleaned:Heinrich Zollinger,2,https://herbarium.bgbm.org/object/B101097046


In [5]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))


----------------------------------------
show names with **particle** found 742 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
21177,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
365,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
4211,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
61043,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
16897,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 15 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17420,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
59080,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
17150,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
807,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
26342,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17252,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [6]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
66765,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Ždanova,O."
32999,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Lomonosova,M., Ždanova,O. & Šaulo,D."
66766,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.","Žíla,V."
66767,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",Волкова Е.
66768,Жирова,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Жирова, O.","Жирова,O."


In [7]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]) # custom function, to get the first entry
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.","A. Cano,E.",1,https://herbarium.bgbm.org/object/B100699397
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",3,https://je.jacq.org/JE00010154
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.","Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.","Morales,J.F. & Abarca,R.",1,https://herbarium.bgbm.org/object/B101153811
...,...,...,...,...,...,...,...,...,...,...,...,...
20802,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelazny, J.","Żelazny,J.",4,https://herbarium.bgbm.org/object/B100344466
20803,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Ždanova,O.",6,https://herbarium.bgbm.org/object/B100263330
20804,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.","Žíla,V.",3,https://herbarium.bgbm.org/object/B100009590
20805,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",Волкова Е.,1,https://herbarium.bgbm.org/object/B100530714


In [8]:
# show collectors with highest occurrenceID_collectors_count
collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family', 'given'], ascending=[False, True, True]).head(10)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
20139,Willing,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Willing, E.","Willing,E. & Eisenblätter,R.",189994,https://herbarium.bgbm.org/object/B100145955
20140,Willing,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Willing, R.","Willing,R. & Willing,R.",188088,https://herbarium.bgbm.org/object/B100074611
7350,Hein,P.,NaN,NaN,NaN,NaN,NaN,NaN,"Hein, P.","Kilian,N., Hein,P. & Oberprieler,C.",10088,https://herbarium.bgbm.org/object/B100113011
9233,Kilian,N.,NaN,NaN,NaN,NaN,NaN,NaN,"Kilian, N.","Hunger,S. & Kilian,N.",9198,https://herbarium.bgbm.org/object/B100003448
11594,Martius,C.F.P.,NaN,NaN,NaN,NaN,NaN,NaN,"Martius, C.F.P.","Martius, C.F.P.",6226,http://id.snsb.info/snsb/collection/472689/581...
15509,Rodríguez,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Rodríguez, D.","Rodríguez,D., Monterrosa,J., Hernández,A. & Ma...",5206,https://herbarium.bgbm.org/object/B100038970
18402,Tempel,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Tempel, E.","Tempel,E.",4561,https://dr.jacq.org/DR073621
11592,Martius,C. F. P. von,NaN,NaN,NaN,NaN,NaN,NaN,"Martius, C. F. P. von","Martius, C.F.P. von (no. Obs. 1490)",4545,http://id.snsb.info/snsb/collection/117775/176...
16413,Schimper,G.W.,NaN,NaN,NaN,NaN,NaN,NaN,"Schimper, G.W.","Schimper, G.W. (no. 880)",4320,http://id.snsb.info/snsb/collection/108223/167...
7247,Haussknecht,H.K.,NaN,NaN,NaN,NaN,NaN,NaN,"Haussknecht, H.K.","Haussknecht, H.K.",4288,http://id.snsb.info/snsb/collection/474055/583...


In [9]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [11]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [12]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))

Show ngram examples:
- simple name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- data from collectors: [' Aa', 'Aai', 'aik', 'iki', 'ki ']
- data from match-test: [' Wa', 'Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' Oh', 'Oh ']
- data from match-test (full name): [' 18', '188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr', 'dr ']


In [10]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))

(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (BGBM) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [13]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: Number of neighbors required for each sample by default for :meth:`kneighbors` queries (originally 5).

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data with %s neighbor sample(s)...' % (len(query_data), n_neighbors))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_source_data', 'namematch_resource_data', 'namematch_distance']
    )

    print('Done')
    return matches


In [14]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
print("Calculate abbrevated names only …")
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

display(matches)

Calculate abbrevated names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.4985511302948 s
Getting nearest neighbours of 20277 data with 5 neighbor sample(s)...
Completed after 135.16855359077454 s
Finding matches build new data frame ...
Building matches done after 135.34273076057434 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,0,"Matsson, L.P.R.","Matsson, L.P.R.",0.00
1,12679,"Hedrén, M.","Hedrén, M.",0.00
2,12685,"Klein, D.","Klein, D.",0.00
3,12691,"Zaharof, E.","Zaharof, E.",0.00
4,12693,"Nyárády, E.J.","Nyárády, E.J.",0.00
...,...,...,...,...
20272,3511,"Lescrauwaet, E.","Lescot, M.",1.22
20273,19644,Kyusoz,"Nyushko, T.I.",1.22
20274,7856,"Avjetijezs, V.","Mekhtijeva, N.A.",1.23
20275,13617,Zulptuwaski,"Załęski, E.",1.24


In [15]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
print("Calculate full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

display(matches_fullnames)


Calculate full names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.44577693939209 s
Getting nearest neighbours of 530 data with 5 neighbor sample(s)...
Completed after 10.70500373840332 s
Finding matches build new data frame ...
Building matches done after 10.709579467773438 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,0,"Buser, Robert","Buser, Robert",0.00
1,361,"Masson, Rosine","Masson, Rosine",0.00
2,357,"Schnedler, Wieland","Schnedler, Wieland",0.00
3,124,"Richter, Karl","Richter, Karl",0.00
4,127,"Hang, Sun","Hang, Sun",0.00
...,...,...,...,...
525,360,"Entry, Provisional","Şentürk, Ozan",1.20
526,447,"Kirinco, Synho","Synge, Hugh",1.20
527,402,"Xinjiang, Inst Bio-Ped-Psamm","Insam, J",1.21
528,201,"Rainbird, Coldecott","Colden, Jane",1.21


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

In [16]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.","A. Cano,E.",1,https://herbarium.bgbm.org/object/B100699397,15093,"A. Cano, E.","Cano, Á.",0.64
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305,4713,Aaiki,"Naiki, A.",0.84
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",3,https://je.jacq.org/JE00010154,6044,"Aaronsohn, A.","Aaronsohn, A.",0.00
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.","Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620,9674,"Abaouz, A.","Arbaoui, S.",1.12
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.","Morales,J.F. & Abarca,R.",1,https://herbarium.bgbm.org/object/B101153811,14101,"Abarca, R.","Abarca, L.",0.57


In [17]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Abdallah, Raffael","Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",1,https://herbarium.bgbm.org/object/B200125981,410,"Abdallah, Raffael","Issel, Raffaele",1.04
1,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,"Abdul, Kadir Bin",Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021,27,"Abdul, Kadir Bin","Kadir, Zaid bin A",1.00
2,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,"Abreu, Guilherme",Guilherme de Abreu (no. 103),1,http://id.snsb.info/snsb/collection/22086/3086...,118,"Abreu, Guilherme","Rau, Guilherme",0.81
3,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"Adá, García","García Adá, Luceño, Rico,E., Romero,T. & Varga...",1,https://herbarium.bgbm.org/object/B100296455,295,"Adá, García","Adá, Ramón García",0.78
4,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers","Ahagen, Schiers,C. & al.",1,https://herbarium.bgbm.org/object/B100194646,22,"Ahagen, Schiers","Séchier, P",1.07


In [18]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",3,https://je.jacq.org/JE00010154,6044,"Aaronsohn, A.","Aaronsohn, A.",0.0
7,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.","Raup,H.M. & Abbe,E.C.",2,https://herbarium.bgbm.org/object/B100241637,3270,"Abbe, E.C.","Abbe, E.C.",0.0
11,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.","Bécquer,E. & Abbott,J.R.",80,https://herbarium.bgbm.org/object/B100181131,15073,"Abbott, J.R.","Abbott, J.R.",0.0
13,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.","Abbott, W.L. (no. 1248)",4,http://id.snsb.info/snsb/collection/504820/626...,7492,"Abbott, W.L.","Abbott, W.L.",0.0
19,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.","Abedin,S. & Hussein,A.",14,https://herbarium.bgbm.org/object/B100046632,1501,"Abedin, S.","Abedin, S.",0.0


In [19]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

In [20]:
# Save the plain name matching results only ...

do_custom_data_aggregation=False
if do_custom_data_aggregation:
    if not os.path.exists('data'):
        print("Make data directory for saving …")
        os.makedirs('data')
    
    # Set some global varialbes
    # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
    this_timestamp_for_data=20230823
    
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_all_matches.to_csv(this_output_file)
    
    print("Wrote plain name matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

### Merge and Aggregate Matched Data and WikiData’s

Review (TODO)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [21]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [22]:
print("Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
print("Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …")
print("Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …")
print("      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)")

criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link'
])

Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)
Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …
Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …
      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,wikidata_link
10703,2,https://dr.jacq.org/DR049432,Kotschy,"Kotschy, T.",0.43,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10704,37,http://id.snsb.info/snsb/collection/16719/2549...,"Kotschy, K.G.T.","Kotschy, T.",0.72,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10705,310,http://id.snsb.info/snsb/collection/117808/176...,"Kotschy, T.","Kotschy, T.",0.00,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10706,5,https://herbarium.bgbm.org/object/B100526350,"Kotschy, Th","Kotschy, T.",0.50,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10707,1,https://herbarium.bgbm.org/object/B100160086,"Kotschyi, C.G.T.","Kotschy, T.",0.92,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10709,1,http://id.snsb.info/snsb/collection/22980/3175...,"Kotschy, C.G.","Kotschy, C.F.",0.63,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
10710,2494,http://id.snsb.info/snsb/collection/108230/167...,"Kotschy, C.G.T.","Kotschy, C.F.",0.76,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
22071,2,https://je.jacq.org/JE00022436,"Kotschy, Carl Georg Theodor","Kotschy, Theodor",0.68,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
22072,1,https://herbarium.bgbm.org/object/B101113772,"Kotschy, Karl Georg Th","Kotschy, Theodor",0.98,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299


Aggregate data to get atomized listings of multiple resource name matches joining by “|” aso.

In [23]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    print('Group data by canonical names (abbreviated and full name): '
          '1st multiple related WD items (e.g. Q1232456), 2nd multiple WD item labels')
    for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
        print('Run %s: Group by wiki data’s %s, and aggregate/join item(s) '
              'by “…|…”, add new column items_joined ...' % (i + 1, wd_matching_column))
        wdata_joined_items = wikidata.groupby([wd_matching_column])['item'].apply('|'.join).reset_index()
        wdata_joined_items.rename(columns={'item': 'items_joined'}, inplace=True)
    
        # print("Done. Show examples of items having multiple matching data «|» … ")
        # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
        # wdata_joined_items[criterion].head()
    
        print('Run %s:   Merge all based on namematch_resource_data, add item(s) data ...' % (i + 1))
        collectors_matches_g2 = pd.merge(
            collectors_matches_g1_merged_wikidata, wdata_joined_items,
            left_on='namematch_resource_data', right_on=wd_matching_column
            , suffixes=('__wikidata_merge', '__grp_by_items')
            # append to left-data, right-data only when identical column names occur
        )
    
        # link wikidata items with canonical name string (pipe separated if more than one)
        print('Run %s:   Group by wiki data’s %s, and aggregate/join itemLabel(s) '
              'by “…|…”, add new column item_labels_joined ...' % (i + 1, wd_matching_column))
        wdata_joined_itemlabels = wikidata.groupby([wd_matching_column])['itemLabel'].apply('|'.join).reset_index()
        wdata_joined_itemlabels.rename(columns={'itemLabel': 'item_labels_joined'}, inplace=True)
    
        print('Run %s:   Merge all based on namematch_resource_data, add itemLabel(s) data ...' % (i + 1))
        collectors_matches_g3 = pd.merge(
            collectors_matches_g2, wdata_joined_itemlabels,
            left_on='namematch_resource_data', right_on=wd_matching_column
            , suffixes=('__grp_by_items', '__grp_by_item_labels')
            # append to left-data, right-data only when identical column names occur
        )
    
        print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
        collectors_matches_group = collectors_matches_g3 \
            if i == 0 \
            else pd.concat([collectors_matches_group, collectors_matches_g3], ignore_index = True)
        
    print('Done')

In [24]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    print("Show examples of item_labels_joined having multiple matching data «|» … ")
    criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)
    
    collectors_matches_group[criterion].get([ # empty 
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
        # 'canonical_string_fullname', 
        'item_labels_joined', 'items_joined'
    ], default="...get: Are data empty or it has probably a wrong named column?")


In [25]:
# check what columns we have and what we would keep for further analysis and what to drop
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    pprint.pprint(collectors_matches_group.columns)
# from merge: _x would means normally from left column, _y means from right column
# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'

Prepare data to save later on …

In [26]:
# Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    collectors_matches_group_simplified = collectors_matches_group.get(
        ['family', 'given', 'canonical_string_collector_parsed', 
          'namematch_source_data', # redundant: 'namematch_source_data' == 'canonical_string_collector_parsed'
         'namematch_resource_data', 'namematch_distance', 
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye',
        'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
        ], default="...get: Are data empty or it has probably a wrong named column?"
    )
    # collectors_matches_group = collectors_matches_g3
    collectors_matches_group_simplified.sort_values(
        by=['namematch_distance', 'canonical_string_collector_parsed']
        , inplace=True
    )
    collectors_matches_group_simplified.drop_duplicates(inplace=True)
    collectors_matches_group_simplified.head()

In [27]:
# old file bgbm_collectors_matches_wikidata_items_group_concat_%s.csv
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_matches_group.to_csv(this_output_file)
    
    print("Wrote groups of collectors matches into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

### Merge Data to Individual WikiData Items

For this, merge by namematch_resource_data and focus to get individual WikiData items.

In [28]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    print('Merge simply namematch_resource_data to Wiki data for abbreviated and full names... ')
    for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
    
        # join wikidata items to avh collectors matches
        #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
        #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
        # link counts of wikidata items with same canonical name string
        #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
        #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
        
        print('Run %s:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...' % (i + 1))
        collectors_matches_wd1 = pd.merge(
            collectors_all_matches, wikidata,
            left_on='namematch_resource_data', right_on=wd_matching_column,
            suffixes=('__coll_all_matches', '__wd')
            # append to left-data, right-data only when identical column names occur
        )
    
        print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
        collectors_matches_with_wdata = collectors_matches_wd1 \
            if i == 0 \
            else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)
    
    print('Done')


In [29]:
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    pprint.pprint(collectors_matches_with_wdata.columns)
    # echo "${text}" | fold --spaces | sed 's@^@#  @'

    collectors_matches_with_wdata.drop_duplicates(inplace=True)
    display(collectors_matches_with_wdata)

Save all columns for further analysis

In [30]:
# old bgbm_collectors_matches_wikidata-botanists_all-columns_%s.csv
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_matches_with_wdata.sort_values(
        by=['namematch_distance', 'canonical_string_collector_parsed']
        , inplace=True
    )
    collectors_matches_with_wdata.to_csv(
        this_output_file, index=False # drop index column
    )
    
    print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

In [31]:
# TODO meaningful?
# remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
    compact_df_tmp.sort_values(
        by=['namematch_distance', 'canonical_string_collector_parsed']
        , inplace=True
    )
    
    # old bgbm_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv
    # results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_%s.csv' % (
        # "20230823"
        time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
    )
    
    compact_df_tmp.to_csv(
        this_output_file, index=False # drop index column
    )
    
    print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

In [32]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(
    by=['namematch_distance', 'family', 'given'], 
    ascending=[True, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossim.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_distance", 
    "source_data", 
    "itemLabel", 
    "item",
    'yob', 'yod'
]).copy()

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_distance_annotation'] = dwcagent_attr_output['namematch_distance'].astype(str).str.replace(r'(.+)', '\\1 (k-means distance)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_distance_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["custom_score_lifetime_data"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & pd.notnull(dwcagent_attr_output["yod"]),
    "custom_score_lifetime_data"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & dwcagent_attr_output["yod"].isnull(),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & pd.notnull(dwcagent_attr_output["yod"]),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & dwcagent_attr_output["yod"].isnull(),
    "custom_score_lifetime_data"
] = 0


# annotations True cases
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & pd.notnull(dwcagent_attr_output["yod"]), 
    "custom_score_lifetime_data_annotation"
] = "life time known"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & dwcagent_attr_output["yod"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & pd.notnull(dwcagent_attr_output["yod"]), 
    "custom_score_lifetime_data_annotation"
] = "year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & dwcagent_attr_output["yod"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "unknown life time"


dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

namematch_distance_max=dwcagent_attr_output['namematch_distance'].max()
dwcagent_attr_output['custom_score_overall'] = (
    # reconsider/transform distance (0 … xx, range larger than 1) to similarity (1 … 0, range of 1) for scoring
    abs( dwcagent_attr_output['namematch_distance'] - namematch_distance_max ) / namematch_distance_max * \
    (
        ( dwcagent_attr_output["custom_score_lifetime_data"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)


dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (data life time score);"
                " {lifetime_score_annote} (data life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_distance_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime_data"],
    lifetime_score_annote=row["custom_score_lifetime_data_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    by=['namematch_distance', 'family', 'given', 'custom_score_overall'], 
    ascending=[True, True, True, False], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
show_display_output=True
if show_display_output:
    display(dwcagent_attr_output.loc[pd.notnull(dwcagent_attr_output['yob'])].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob'].isnull()].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))


/tmp/ipykernel_33209/913966788.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,custom_score_lifetime_data,custom_score_lifetime_data_annotation
4,A. Aaronsohn,Aaron Aaronsohn,0.50,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1876-1919,1.0,life time known
10,E.C. Abbe,Ernst Cleveland Abbe,0.50,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1905-2000,1.0,life time known
14,J.R. Abbott,J. Richard Abbott,0.25,0.0 (k-means distance); 0.25 (score overall); ...,0.0,0.0,1968-?,0.5,year of death is missing
16,W.L. Abbott,William Louis Abbott,0.50,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1860-1936,1.0,life time known
26,S. Abedin,Sultanul Abedin,0.25,0.0 (k-means distance); 0.25 (score overall); ...,0.0,0.0,1952-?,0.5,year of death is missing


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,custom_score_lifetime_data,custom_score_lifetime_data_annotation
66,M. Ackermann,Marianne Ackermann,-0.25,0.0 (k-means distance); -0.25 (score overall);...,-0.5,0.0,?-?,0.0,unknown life time
67,M. Ackermann,Manfred Ackermann,-0.25,0.0 (k-means distance); -0.25 (score overall);...,-0.5,0.0,?-?,0.0,unknown life time
76,J.M. Acosta,Juan M. Acosta,0.00,0.0 (k-means distance); 0.00 (score overall); ...,0.0,0.0,?-?,0.0,unknown life time
77,L. Acosta,L. Acosta,0.00,0.0 (k-means distance); 0.00 (score overall); ...,0.0,0.0,?-?,0.0,unknown life time
125,W. Adler,Wilhelm Adler,-0.25,0.0 (k-means distance); -0.25 (score overall);...,-0.5,0.0,?-?,0.0,unknown life time


In [33]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
# refactor custom_namematch_similarity      → custom_namematch_distance
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'namematch_distance':                 'custom_namematch_distance'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    dwcagent_attr_output.head(20)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 … represents the available data quality not the match in the first place
        'name',
        'attributionRemarks',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_distance',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)


In [34]:
show_display_output=True
if show_display_output:
    criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda x: x < 0 )
    
    display(dwcagent_attr_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_distance,custom_score_multiple_names,custom_score_lifetime_data
65,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,1,Markus Ackermann,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.00,0.0,-0.5,0.5
66,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,2,Marianne Ackermann,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,0.0,-0.5,0.0
67,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,3,Manfred Ackermann,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,0.0,-0.5,0.0
126,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,1,Wolfgang Adler,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q119062956,0.00,0.0,-0.5,0.5
125,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,2,Wilhelm Adler,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q10393599,-0.25,0.0,-0.5,0.0
189,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,1,Sultan Ahmad,0.0 (k-means distance); 0.25 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112765,0.25,0.0,-0.5,1.0
188,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,2,Shamsuddin Ahmad,0.0 (k-means distance); 0.00 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q33661104,0.00,0.0,-0.5,0.5
190,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,3,Suhail Ahmad,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q88848655,-0.25,0.0,-0.5,0.0
282,http://id.snsb.info/snsb/collection/459414/560...,"Albo, G.",G. Albo,1,Giacomo Albo,0.0 (k-means distance); 0.25 (score overall); ...,Person,collected,wikidata,http://www.wikidata.org/entity/Q21505027,0.25,0.0,-0.5,1.0
281,http://id.snsb.info/snsb/collection/459414/560...,"Albo, G.",G. Albo,2,Giacomo Albo,0.0 (k-means distance); -0.25 (score overall);...,Person,collected,wikidata,http://www.wikidata.org/entity/Q10289422,-0.25,0.0,-0.5,0.0


In [35]:
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20230823
this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_dwc-agent-output_20230823.csv (7718 kB)


## Documentation

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest